## 將資料儲存至資料庫

### 下載輿情資料

In [1]:
! wget https://raw.githubusercontent.com/ywchiu/sns_mining/main/data/ptt_creditcard.xlsx

--2023-04-14 13:26:21--  https://raw.githubusercontent.com/ywchiu/sns_mining/main/data/ptt_creditcard.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 479425 (468K) [application/octet-stream]
Saving to: ‘ptt_creditcard.xlsx’

ptt_creditcard.xlsx 100%[===================>] 468.19K  --.-KB/s    in 0.007s  

2023-04-14 13:26:22 (70.0 MB/s) - ‘ptt_creditcard.xlsx’ saved [479425/479425]



### 讀取輿情資料

In [4]:
import pandas
data_df = pandas.read_excel('ptt_creditcard.xlsx', index_col = 0)
data_df.head()

,title,link,author,push,date,content
0,Re: [情報] 2023 凱基行動支付1.5%/4% 悠遊卡速食8%活動,https://www.ptt.cc/bbs/creditcard/M.1681312432...,thomson,9,4/12,放榜了\n凱基魔Buy+橘子支付 繳費有4%回饋\n凱基魔Buy+全盈PAY 繳費有 1~3...
1,[心得] 台新 GoGo&Flygo核卡,https://www.ptt.cc/bbs/creditcard/M.1681343426...,peter120,5,4/13,★職業類別：通訊產業\n\n★年資：1.5y\n\n★年齡：25\n\n★申請卡片：@GoG...
2,[閒聊] 花旗卡盜刷,https://www.ptt.cc/bbs/creditcard/M.1681358049...,t1013tom,13,4/13,平常都在板上潛水看大神們分享的攻略\n每次看到盜刷文都覺得不太乾我的事，因為我比較少用網拍買...
3,[心得] 兆豐新光核卡 渣打花旗婉拒 多家調額,https://www.ptt.cc/bbs/creditcard/M.1681373559...,yeuling9300,NaN,4/13,職業類別：出版業\n年資：2y\n年齡：29\n申請卡片：兆豐宇宙明星BT21信用卡/新光O...
4,[心得] 富邦好市多鈦金相片卡核卡,https://www.ptt.cc/bbs/creditcard/M.1681374700...,gyqmo,7,4/13,★職業類別：教育業\n\n★年資：10\n\n★年齡：37\n\n★申請卡片：富邦好市多相片...


### 資料前處理

In [8]:
data_df['date'] = '2023/' +  data_df['date'] 
data_df['date'] = pandas.to_datetime(data_df['date'], format='%Y/ %m/%d')

### 將資料匯入 SQLite

In [9]:
import sqlite3 as lite
with lite.connect('sns_data.sqlite') as db:
 data_df.to_sql(name='main_content', index=False, con=db, if_exists='replace')


### 用SQL 讀取資料庫資料

In [10]:
with lite.connect('sns_data.sqlite') as con:
 out_df = pandas.read_sql_query(
   'select date, count(*) from main_content GROUP BY date ORDER by date desc', con = con)

In [11]:
out_df

,date,count(*)
0,2023-04-14 00:00:00,2
1,2023-04-13 00:00:00,11
2,2023-04-12 00:00:00,11
3,2023-04-11 00:00:00,4
4,2023-04-10 00:00:00,5
...,...,...
92,2023-01-11 00:00:00,6
93,2023-01-10 00:00:00,8
94,2023-01-09 00:00:00,18
95,2023-01-08 00:00:00,7


## 繪製輿情資料

### 聲量分析

In [12]:
import plotly.express as px
fig = px.line(out_df, x="date", y="count(*)", title='輿情聲量')
fig.show()

In [13]:
with lite.connect('sns_data.sqlite') as con:
 # 取得玉山資料
 out_df1 = pandas.read_sql_query(
   '''SELECT date, count(*) FROM main_content
   WHERE content LIKE '%玉山%'
   GROUP BY date ORDER by date desc''', con = con)
 
 # 取得中信資料
 out_df2 = pandas.read_sql_query(
   '''SELECT date, count(*) FROM main_content
   WHERE content LIKE '%中信%'
   GROUP BY date ORDER by date desc''', con = con)

 ### 聲量比較

In [14]:
import plotly.express as px
fig1 = px.line(out_df1, x="date", y="count(*)", title='輿情聲量')
fig2 = px.line(out_df2, x="date", y="count(*)")
fig1.update_traces(line=dict(color='blue'))
fig2.update_traces(line=dict(color='red'))
fig1.add_trace(fig2.data[0])
fig1.show()

### 比較主文與評論

In [15]:
with lite.connect('sns_data.sqlite') as con:
 out_df1 = pandas.read_sql_query(
   '''SELECT date, count(*) as article_count, sum(push) as push_count FROM main_content
   GROUP BY date ORDER by date desc''', con = con)
out_df1

,date,article_count,push_count
0,2023-04-14 00:00:00,2,22.0
1,2023-04-13 00:00:00,11,90.0
2,2023-04-12 00:00:00,11,157.0
3,2023-04-11 00:00:00,4,174.0
4,2023-04-10 00:00:00,5,89.0
...,...,...,...
92,2023-01-11 00:00:00,6,93.0
93,2023-01-10 00:00:00,8,179.0
94,2023-01-09 00:00:00,18,230.0
95,2023-01-08 00:00:00,7,90.0


In [18]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=("主文", "評論數"))

fig.add_trace(go.Scatter(x=out_df1['date'], y=out_df1['article_count'], name='主文'), row=1, col=1)
fig.update_yaxes(range=[0, 30], row=1, col=1)

fig.add_trace(go.Bar(x=out_df1['date'], y=out_df1['push_count'], name='評論數'), row=2, col=1)
fig.update_yaxes(range=[0, 400], row=2, col=1)

fig.update_layout(height=600, title_text="主文與評論數聲量報表", showlegend=False)
fig.show()


## 運用InfoMiner 輿情資料

In [78]:
! wget https://raw.githubusercontent.com/ywchiu/sns_mining/main/data/InfoMiner_%E4%BF%A1%E7%94%A8%E5%8D%A1.csv

--2023-04-14 14:15:00--  https://raw.githubusercontent.com/ywchiu/sns_mining/main/data/InfoMiner_%E4%BF%A1%E7%94%A8%E5%8D%A1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6841678 (6.5M) [text/plain]
Saving to: ‘InfoMiner_信用卡.csv.1’

InfoMiner_信用卡.cs 100%[===================>]   6.52M  --.-KB/s    in 0.02s   

2023-04-14 14:15:00 (283 MB/s) - ‘InfoMiner_信用卡.csv.1’ saved [6841678/6841678]



In [79]:
import pandas
data_df = pandas.read_csv('InfoMiner_信用卡.csv', encoding='utf-8', parse_dates = ['date']  )
data_df.head(3)

,id,url,title,content,push_content,region,type,source,ctype,date,...,share_count,comment_count,like_count,dislike_count,positive_pushs,neutual_pushs,negative_pushs,sentiment,crawled_at,hit_rate
0,2490c1ac-c9e7-563f-a09d-d5ca3216aaeb,https://www.ptt.cc/bbs/creditcard/M.1681470237...,永豐大戶卡請益,最近辦了人生第二張信用卡，昨天跟爸爸討論後才發現他的永豐信用卡回饋只有0.666%比我目前辦...,NaN,台灣,ptt 政治,批踢踢實業坊,creditcard,2023-04-14,...,0.0,0.0,0,0,0.0,0.0,0.0,NaN,NaN,NaN
1,8e9121e4-01cf-5e93-8514-a095e256ca45,https://www.facebook.com/147209485395938/posts...,NaN,臺灣Pay幫你省油費😎自駕出遊輕鬆玩也能輕鬆省～⛽福懋加油站👉 https://tw-pay...,NaN,台灣,綠營側翼,臉書,臺灣Pay,2023-04-14,...,0.0,0.0,1,0,0.0,0.0,0.0,NaN,NaN,NaN
2,ed8de5bf-6312-5f2c-b47d-c350bd37da04,https://www.ettoday.net/news/20230414/2478241.htm,退休金存多少才夠用？5步驟完整規劃退休理財 善用「4%法則」輕鬆試算,文／Workworks職場人想提早退休，那麼該提前準備多少退休金呢？職場人們有曾經預想過未來...,NaN,台灣,新聞,Ettoday新聞雲,ETtoday新聞雲,2023-04-14,...,0.0,0.0,0,0,0.0,0.0,0.0,NaN,NaN,NaN


In [80]:
import sqlite3 as lite
with lite.connect('sns_data.sqlite') as db:
 data_df.to_sql(name='infominer_creditcard_content', index=False, con=db, if_exists='replace')


In [81]:
with lite.connect('sns_data.sqlite') as con:
 out_df = pandas.read_sql_query(
   'select date, count(*) from infominer_creditcard_content WHERE content IS NOT NULL GROUP BY date ORDER by date desc', con = con)

In [82]:
out_df

,date,count(*)
0,2023-04-14 00:00:00,245
1,2023-04-13 00:00:00,359
2,2023-04-12 00:00:00,383
3,2023-04-11 00:00:00,352
4,2023-04-10 00:00:00,429
5,2023-04-09 00:00:00,157
6,2023-04-08 00:00:00,271
7,2023-04-07 00:00:00,332


In [83]:
import plotly.express as px
fig = px.line(out_df, x="date", y="count(*)", title='輿情聲量')
fig.show()

## 練習題
請運用模仿犯資料畫出聲量趨勢圖
- https://raw.githubusercontent.com/ywchiu/sns_mining/main/data/InfoMiner_%E6%A8%A1%E4%BB%BF%E7%8A%AF.csv

In [91]:
! wget https://raw.githubusercontent.com/ywchiu/sns_mining/main/data/InfoMiner_%E6%A8%A1%E4%BB%BF%E7%8A%AF.csv

--2023-04-14 14:22:47--  https://raw.githubusercontent.com/ywchiu/sns_mining/main/data/InfoMiner_%E6%A8%A1%E4%BB%BF%E7%8A%AF.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4994714 (4.8M) [text/plain]
Saving to: ‘InfoMiner_模仿犯.csv’

InfoMiner_模仿犯.cs 100%[===================>]   4.76M  --.-KB/s    in 0.02s   

2023-04-14 14:22:47 (245 MB/s) - ‘InfoMiner_模仿犯.csv’ saved [4994714/4994714]



In [92]:
import pandas
data_df = pandas.read_csv('InfoMiner_模仿犯.csv', encoding='utf-8', parse_dates = ['date']  )
data_df.head(3)

,id,url,title,content,push_content,region,type,source,ctype,date,...,share_count,comment_count,like_count,dislike_count,positive_pushs,neutual_pushs,negative_pushs,sentiment,crawled_at,hit_rate
0,de1308c6-9bea-53f4-8f20-83762a6a2ebd,http://news.pchome.com.tw/entertainment/nownew...,王淨辣曬螞蟻腰！他昏倒女神幫CPR遭嫌棄,王淨（左）與同理合唱歌曲〈Beautiful Night〉，還辣曬螞蟻腰參與演出MV。（圖／...,NaN,台灣,新聞,PChome 新聞,娛樂,2023-04-14,...,0.0,0.0,0,0,0.0,0.0,0.0,NaN,NaN,NaN
1,a76ae8d9-f7dc-5ecd-a2b1-465d22f83f54,https://tw.news.yahoo.com/%E5%A7%9A%E6%B7%B3%E...,姚淳耀扯女神頭喊享受 林心如放心讓他殺,Netflix現正熱播的華語懸疑影集《模仿犯》自3/31全球上線後，便引起廣大討論，更登上N...,NaN,台灣,新聞,Yahoo,娛樂,2023-04-14,...,0.0,0.0,0,0,0.0,0.0,0.0,NaN,NaN,NaN
2,334f636f-e98b-509d-b1c8-1db2eac0f720,https://www.facebook.com/144589512304563/posts...,NaN,雖然知道是特效但還是覺得看了脖子痛痛der！！！（羊羹）💡星光雲LINE掌握明星大小事：ht...,NaN,台灣,綠營側翼,臉書,ETtoday星光雲,2023-04-14,...,0.0,0.0,17,0,0.0,0.0,0.0,NaN,NaN,NaN
